From earlier experiments it became clear the performance of ICD coding using GPT-3.5 and GPT-4 is mediocre
Should the prompting be improved or is there another cause?
Lets validate if GPT can reproduce the ICD-10 code descriptions

In [2]:
import pandas as pd

In [3]:
!pip install promptify



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [9]:
header = ["doc", "code"]

diagnoses = set()
procedures = set()
for split in ["train", "dev", "test"]:
    df = pd.read_csv("codiesp/final_dataset_v4_to_publish/{split}/{split}D.tsv".format(dir=dir, split=split), delimiter="\t", names=header)
    diagnoses.update(df["code"].unique())
    df = pd.read_csv("codiesp/final_dataset_v4_to_publish/{split}/{split}P.tsv".format(dir=dir, split=split), delimiter="\t", names=header)
    procedures.update(df["code"].unique())

print(len(diagnoses))
print(len(procedures))

2557
870


In [10]:
import random

random_sampled_diagnoses = random.sample(list(diagnoses), 100)
random_sampled_procedures = random.sample(list(procedures), 100)

In [11]:
print(random_sampled_diagnoses)

['h05.233', 'q51.811', 'n26.9', 'z82.41', 's23.163', 'b96.89', 'r74.0', 'h21.309', 'a04.7', 'n85.9', 'n41.0', 'h35.349', 'c22.1', 'h53.142', 'n17.0', 'g36.0', 'b59', 'h04.209', 'k31.1', 'l58.9', 'h93.19', 'c53.9', 'e85.4', 'l88', 'i15.9', 's22.49xa', 'k31.9', 'r91.8', 'r04.89', 'g24.5', 'm25.531', 'm54.9', 'z79.4', 'h16.449', 'm27.49', 'c63.8', 'i48.2', 'd72.819', 'z3a.19', 't20.04x', 'b58.2', 'r61', 'r40.2412', 'n05.1', 'm08.961', 'd47.9', 'k56.69', 's06.2x', 'z87.440', 'q62.4', 'h20.05', 'i71.00', 'k57.20', 'f45.29', 'r63.3', 'h50.012', 'r06.09', 'l93.0', 'm70.51', 'h35.62', 'k50.00', 'i87.2', 'm43.6', 'd37.8', 'a01.00', 'h01.8', 'q18.0', 'm48.00', 'a15.9', 'i70.0', 'n83.9', 'a54.9', 'i63.519', 'k14.8', 'l50.9', 'q07.8', 'm79.643', 'h53.47', 'h53.462', 'n18.2', 'n45.2', 'm86.8x5', 'i72.9', 's71.101', 's22.5xx', 'c83.0', 't17.810', 'r27.8', 'j47.9', 's61.209', 'b67.8', 'm41.9', 'l80', 'l25.9', 'i48.0', 'e27.40', 'q76.49', 'z68.42', 'g24.9', 'b33.3']


In [12]:
n = 10
random_sampled_diagnoses_chunked = [random_sampled_diagnoses[i:i+n] for i in range(0, len(random_sampled_diagnoses), n)]        # Use xrange in py2k
random_sampled_procedures_chunked = [random_sampled_procedures[i:i+n] for i in range(0, len(random_sampled_procedures), n)]        # Use xrange in py2k


In [ ]:
print(random_sampled_diagnoses_chunked)

[['i82.619', 'h57.9', 'd73.89', 'h47.10', 'q87.3', 'n60.1', 'm79.60', 'h30.89', 'f19.20', 'r86.9'], ['b95.8', 's70.32', 'k31.89', 'h49.11', 'n76.5', 'i61.3', 'l94.3', 'h57.8', 'j82', 'h47.20'], ['k06.2', 'q38.2', 'r10.11', 'n23', 'n28.1', 'h54.61', 'b58.9', 'i12.0', 'r40.2412', 'q04.3'], ['i77.71', 'a18.10', 'a18.50', 'w56.81x', 'h35.62', 'c79.82', 's32.10', 'b34.9', 'n97.9', 'q87.89'], ['a49.9', 'k62.0', 'g83.24', 'c49.a2', 'k57.30', 'n44.8', 'j85.2', 'f12.20', 'e85.9', 'h18.2'], ['m26.02', 'n94.89', 'r91.1', 'e83.42', 'q60.5', 'c49.a', 'k62.9', 'i85.00', 'g60.8', 'm25.439'], ['r39.13', 'm85.00', 'f70', 'k22.70', 's49.92x', 'r09.2', 'k04.7', 'd30.3', 'q74.0', 'd48.1'], ['w19.xxx', 't20.30x', 'k72.9', 'a41.81', 's05.8x2', 'p83.8', 'z17.1', 'k13.0', 'm79.1', 'z74.01'], ['f05', 'd72.820', 'l29.0', 'z79.810', 'j84.10', 'n32.0', 'e83.51', 'z90.710', 'k63.5', 'h01.001'], ['h50.011', 'c90.00', 'd15.1', 'r20.0', 'n28.9', 'k11.1', 't85.41x', 'm62.3', 'b19.20', 'h21.9']]


In [15]:
def get_prompt_description(codes):
    output = [{"ICD-10-code": "code", "Description": "the description"}]
    prompt1 = """Add for each code ICD-10 the correct description as would be found. The input is a list of ICD-10 codes.
Output: {output}

Input: {codes}
Output: """.format(output=output, codes=codes)
    return prompt1

print(get_prompt_description(random_sampled_diagnoses_chunked[0]))

Add for each code ICD-10 the correct description as would be found. The input is a list of ICD-10 codes.
Output: [{'ICD-10-code': 'code', 'Description': 'the description'}]

Input: ['h05.233', 'q51.811', 'n26.9', 'z82.41', 's23.163', 'b96.89', 'r74.0', 'h21.309', 'a04.7', 'n85.9']
Output: 


In [31]:
df_diag = pd.DataFrame()
df_diag["diag"] = random_sampled_diagnoses_chunked

print(df_diag)


                                                diag
0  [i75.89, h00.11, k92.0, m35.9, i85.01, s25.02x...
1  [j82, r47.02, e11.64, e23.2, s70.312, l00, b46...
2  [e80.4, m54.31, e87.1, r06.4, c43.9, m80.88x, ...
3  [h31.309, i44.2, h05.20, f60.7, k83.8, n30.80,...
4  [k56.2, h50.21, r01.0, k42.9, m26.619, d53.1, ...
5  [d17.5, h11.002, c85.9, c66.1, h11.422, c78.7,...
6  [d86.9, r19.8, f32.3, d37.8, g20, i35.8, s06.9...
7  [a63.0, d73.4, c83.39, c53.9, i88.1, i48.92, d...
8  [i63.511, l02.41, h21.42, k52.82, m87.059, s39...
9  [d40.11, e75.5, d44.4, s53.101, b10.89, h30.14...


In [35]:
prompts  = []
for chunk in random_sampled_diagnoses_chunked:
    prompts.append(get_prompt_description(chunk))
print(len(prompts))
df_diag["prompt_descr"] = prompts

df_diag.to_excel('resources/prompts-diagnoses.xlsx')

10


Now use GPT-4: https://chat.openai.com/?model=gpt-4 set response to col "response_descr" and overwrite the file

In [37]:
df_diag = pd.read_excel('resources/prompts-diagnoses.xlsx')
df_diag.head()

,Unnamed: 0,diag,prompt_descr,response_descr
0,0,"['i75.89', 'h00.11', 'k92.0', 'm35.9', 'i85.01...",Add for each code ICD-10 the correct descripti...,"[ {""ICD-10-code"": ""i75.89"", ""Description"": ..."
1,1,"['j82', 'r47.02', 'e11.64', 'e23.2', 's70.312'...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""j82"", ""Description"": ""Pu..."
2,2,"['e80.4', 'm54.31', 'e87.1', 'r06.4', 'c43.9',...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""e80.4"", ""Description"": ..."
3,3,"['h31.309', 'i44.2', 'h05.20', 'f60.7', 'k83.8...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""h31.309"", ""Description""..."
4,4,"['k56.2', 'h50.21', 'r01.0', 'k42.9', 'm26.619...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""k56.2"", ""Description"": ..."


The following prompt is used

Evaluate the PREDICTED to the ACTUAL description in the json, the prediction does not have to be strictly equal, but have the same meaning. Add a field EVAL to the json for the boolean outcome. Here is the json:

In [ ]:
import requests
import json


def get_code_description(icd_10_code):
    request_url = "https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search?df=code,name&authenticity_token=&terms={icd_code}".format(icd_code=icd_10_code)
    x = requests.get(request_url)
    print(x.content)
    json_res = json.loads(x.content)
    print(json_res)
    description = json_res[3][0][1] 
    return description

print(get_code_description(icd_10_code="m86.8x5"))


b'[1,["M86.8X5"],null,[["M86.8X5","Other osteomyelitis, thigh"]]]'
[1, ['M86.8X5'], None, [['M86.8X5', 'Other osteomyelitis, thigh']]]
Other osteomyelitis, thigh


In [52]:
#add correct description
official_batches = []
for row in df_diag["response_descr"].to_list():
    json_resp = json.loads(row)
    row_official = []
    for item in json_resp:
        new_item = {}
        new_item['ICD-10-code'] = item["ICD-10-code"]
        new_item['Predicted'] = item["Description"]
        new_item["Actual"] = get_code_description(item["ICD-10-code"])
        row_official.append(new_item)
    official_batches.append(row_official)
df_diag["official"] = official_batches


b'[1,["I75.89"],null,[["I75.89","Atheroembolism of other site"]]]'
[1, ['I75.89'], None, [['I75.89', 'Atheroembolism of other site']]]
b'[1,["H00.11"],null,[["H00.11","Chalazion right upper eyelid"]]]'
[1, ['H00.11'], None, [['H00.11', 'Chalazion right upper eyelid']]]
b'[1,["K92.0"],null,[["K92.0","Hematemesis"]]]'
[1, ['K92.0'], None, [['K92.0', 'Hematemesis']]]
b'[1,["M35.9"],null,[["M35.9","Systemic involvement of connective tissue, unspecified"]]]'
[1, ['M35.9'], None, [['M35.9', 'Systemic involvement of connective tissue, unspecified']]]
b'[1,["I85.01"],null,[["I85.01","Esophageal varices with bleeding"]]]'
[1, ['I85.01'], None, [['I85.01', 'Esophageal varices with bleeding']]]
b'[3,["S25.02XA","S25.02XD","S25.02XS"],null,[["S25.02XA","Major laceration of thoracic aorta, initial encounter"],["S25.02XD","Major laceration of thoracic aorta, subsequent encounter"],["S25.02XS","Major laceration of thoracic aorta, sequela"]]]'
[3, ['S25.02XA', 'S25.02XD', 'S25.02XS'], None, [['S25.02X

In [53]:
print(df_diag.head())

   Unnamed: 0                                               diag  \
0           0  ['i75.89', 'h00.11', 'k92.0', 'm35.9', 'i85.01...   
1           1  ['j82', 'r47.02', 'e11.64', 'e23.2', 's70.312'...   
2           2  ['e80.4', 'm54.31', 'e87.1', 'r06.4', 'c43.9',...   
3           3  ['h31.309', 'i44.2', 'h05.20', 'f60.7', 'k83.8...   
4           4  ['k56.2', 'h50.21', 'r01.0', 'k42.9', 'm26.619...   

                                        prompt_descr  \
0  Add for each code ICD-10 the correct descripti...   
1  Add for each code ICD-10 the correct descripti...   
2  Add for each code ICD-10 the correct descripti...   
3  Add for each code ICD-10 the correct descripti...   
4  Add for each code ICD-10 the correct descripti...   

                                      response_descr  \
0  [    {"ICD-10-code": "i75.89", "Description": ...   
1  [\n  {"ICD-10-code": "j82", "Description": "Pu...   
2   [\n  {"ICD-10-code": "e80.4", "Description": ...   
3   [\n  {"ICD-10-code": "h31.

In [54]:
def get_prompt_eval(official):
    prompt = """Evaluate the "Predicted" with the "Actual" code description in the json. "Predicted" and "Actual" do need to be strictly equal, but the meaning has to be similar. Add a field "Eval" to the json for the boolean outcome and a field "Confidence" for the confidence that your evaluation is correct ["LOW", "MED", "HIGH"]. Here is the json: """ + str(official)
    return prompt

print(get_prompt_eval(df_diag["official"].iloc[0]))

Evaluate the "Predicted" with the "Actual" code description in the json. "Predicted" and "Actual" do need to be strictly equal, but the meaning has to be similar. Add a field "Eval" to the json for the boolean outcome and a field "Confidence" for the confidence that your evaluation is correct ["LOW", "MED", "HIGH"]. Here is the json: [{'ICD-10-code': 'i75.89', 'Predicted': 'Other specified atherosclerosis', 'Actual': 'Atheroembolism of other site'}, {'ICD-10-code': 'h00.11', 'Predicted': 'Primary chalazion right lower eyelid', 'Actual': 'Chalazion right upper eyelid'}, {'ICD-10-code': 'k92.0', 'Predicted': 'Hematemesis', 'Actual': 'Hematemesis'}, {'ICD-10-code': 'm35.9', 'Predicted': 'Systemic involvement of connective tissue, unspecified', 'Actual': 'Systemic involvement of connective tissue, unspecified'}, {'ICD-10-code': 'i85.01', 'Predicted': 'Esophageal varices with bleeding', 'Actual': 'Esophageal varices with bleeding'}, {'ICD-10-code': 's25.02x', 'Predicted': 'Injury of thoraci

In [55]:
df_diag["prompt_eval"] = df_diag["official"].apply(get_prompt_eval)

In [56]:
df_diag.to_excel('resources/prompts-diagnoses.xlsx')

In [58]:
df_diag = pd.read_excel('resources/prompts-diagnoses.xlsx')

In [59]:
df_diag.head()

,Unnamed: 0.1,Unnamed: 0,diag,prompt_descr,response_descr,official,prompt_eval,response_eval
0,0,0,"['i75.89', 'h00.11', 'k92.0', 'm35.9', 'i85.01...",Add for each code ICD-10 the correct descripti...,"[ {""ICD-10-code"": ""i75.89"", ""Description"": ...","[{'ICD-10-code': 'i75.89', 'Predicted': 'Other...","Evaluate the ""Predicted"" with the ""Actual"" cod...","[\n {\n ""ICD-10-code"": ""i75.89"",\n ..."
1,1,1,"['j82', 'r47.02', 'e11.64', 'e23.2', 's70.312'...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""j82"", ""Description"": ""Pu...","[{'ICD-10-code': 'j82', 'Predicted': 'Pulmonar...","Evaluate the ""Predicted"" with the ""Actual"" cod...","[\n {\n ""ICD-10-code"": ""j82"",\n ..."
2,2,2,"['e80.4', 'm54.31', 'e87.1', 'r06.4', 'c43.9',...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""e80.4"", ""Description"": ...","[{'ICD-10-code': 'e80.4', 'Predicted': 'Gilber...","Evaluate the ""Predicted"" with the ""Actual"" cod...","[\n {\n ""ICD-10-code"": ""e80.4"",\n ..."
3,3,3,"['h31.309', 'i44.2', 'h05.20', 'f60.7', 'k83.8...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""h31.309"", ""Description""...","[{'ICD-10-code': 'h31.309', 'Predicted': 'Unsp...","Evaluate the ""Predicted"" with the ""Actual"" cod...","[\n {\n ""ICD-10-code"": ""h31.309"",\n ..."
4,4,4,"['k56.2', 'h50.21', 'r01.0', 'k42.9', 'm26.619...",Add for each code ICD-10 the correct descripti...,"[\n {""ICD-10-code"": ""k56.2"", ""Description"": ...","[{'ICD-10-code': 'k56.2', 'Predicted': 'Volvul...","Evaluate the ""Predicted"" with the ""Actual"" cod...","[\n {\n ""ICD-10-code"": ""k56.2"",\n ..."


In [63]:
reponses = df_diag["response_eval"].to_list()
responses_json = [json.loads(resp) for resp in reponses]
responses_json_flat = [item for sublist in responses_json for item in sublist]

print(responses_json_flat)


[{'ICD-10-code': 'i75.89', 'Predicted': 'Other specified atherosclerosis', 'Actual': 'Atheroembolism of other site', 'Eval': False, 'Confidence': 'HIGH'}, {'ICD-10-code': 'h00.11', 'Predicted': 'Primary chalazion right lower eyelid', 'Actual': 'Chalazion right upper eyelid', 'Eval': False, 'Confidence': 'HIGH'}, {'ICD-10-code': 'k92.0', 'Predicted': 'Hematemesis', 'Actual': 'Hematemesis', 'Eval': True, 'Confidence': 'HIGH'}, {'ICD-10-code': 'm35.9', 'Predicted': 'Systemic involvement of connective tissue, unspecified', 'Actual': 'Systemic involvement of connective tissue, unspecified', 'Eval': True, 'Confidence': 'HIGH'}, {'ICD-10-code': 'i85.01', 'Predicted': 'Esophageal varices with bleeding', 'Actual': 'Esophageal varices with bleeding', 'Eval': True, 'Confidence': 'HIGH'}, {'ICD-10-code': 's25.02x', 'Predicted': 'Injury of thoracic aorta, initial encounter', 'Actual': 'Major laceration of thoracic aorta, initial encounter', 'Eval': False, 'Confidence': 'HIGH'}, {'ICD-10-code': 'h43

In [66]:
df = pd.json_normalize(responses_json_flat)
df.head()

,ICD-10-code,Predicted,Actual,Eval,Confidence
0,i75.89,Other specified atherosclerosis,Atheroembolism of other site,False,HIGH
1,h00.11,Primary chalazion right lower eyelid,Chalazion right upper eyelid,False,HIGH
2,k92.0,Hematemesis,Hematemesis,True,HIGH
3,m35.9,"Systemic involvement of connective tissue, uns...","Systemic involvement of connective tissue, uns...",True,HIGH
4,i85.01,Esophageal varices with bleeding,Esophageal varices with bleeding,True,HIGH


In [68]:
df['Eval'].describe()

count      100
unique       2
top       True
freq        52
Name: Eval, dtype: object

In [69]:
df['Confidence'].describe()

count      100
unique       2
top       HIGH
freq        88
Name: Confidence, dtype: object

In [71]:
df.to_excel("resources/gpt-descriptions/diagnoses_stats.xlsx")